In [1]:
import cv2 as cv
from ultralytics import YOLO
import cvzone
import matplotlib.pyplot as plt
import numpy as np

In [2]:
classes_dict =  {
            '0':'0',
            '1':'1',
            '2':'2',
            '3':'3',
            '4':'4',
            '5':'5',
            '6':'6',
            '7':'7',
            '8':'8',
            '9':'9',
            '10':'b',
            '11':'C',
            '12':'d',
            '13':'G',
            '14':'H',
            '15':'J',
            '16':'l',
            '17':'m',
            '18':'n',
            '19':'S',
            '20':'T',
            '21':'v',
            '22':'y'
            }

In [3]:
plate_detector_model = YOLO('plate_detector.pt')
digits_detector_model = YOLO('digits_detector.pt')

In [4]:
def find_plate(image , information):
    for r in information:
        if len(r.obb.data) != 0:
            for box in r.obb.data:
                x_center, y_center, width, height, angle, confidence, class_id = box
                x_center, y_center, width, height, angle = float(x_center), float(y_center), float(width), float(height), float(angle)

                rect = ((x_center, y_center), (width, height), np.degrees(angle))
                box_points = cv.boxPoints(rect)
                box_points = np.intp(box_points)

                #cv.drawContours(image, [box_points], 0, (0, 255, 0), 5)
                plate_image = image[int(y_center) - int(height / 2) : int(y_center) + int(height / 2) , int(x_center) - int(width / 2) : int(x_center) + int(width / 2) , ...]
                return plate_image , True
        else:
            return None , False


In [5]:
def detect_digits(image , information):
    digits_dict = {}
    for r in information:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            cv.rectangle(image, (x1, y1) ,(x2, y2), (0, 255, 0), 2)
            cls = int(box.cls[0])
            #cv.putText(image , classes_dict[str(cls)] , (x1 , y1 + (y2 // 2)) , cv.FONT_HERSHEY_COMPLEX , 0.5 , (255,0,255) , 2)
            digits_dict[x1] = classes_dict[str(cls)]
    return image , digits_dict

In [ ]:
pelak = cv.imread('plate.png')
pelak = cv.cvtColor(pelak , cv.COLOR_BGR2RGB)
pelak = cv.resize(pelak , dsize=(250 , 60))

video = cv.VideoCapture(0)
while True:
    ok , frame = video.read()
    if not ok:
        break
    
    frame = cv.cvtColor(frame , cv.COLOR_BGR2RGB)

    plate_detected = plate_detector_model(frame , conf=0.5)

    plate_detected_image , situation = find_plate(frame , plate_detected)
    if situation:
        digits_detected = digits_detector_model(plate_detected_image , conf=0.2)
        digits_detected_image , digits_dict = detect_digits(plate_detected_image , digits_detected)
        x = 27
        y = 40
        frame[0:pelak.shape[0] , 0:pelak.shape[1] , ...] = pelak
        for i in sorted(digits_dict):
            cv.putText(frame , digits_dict[i] , (x  , y) , cv.FONT_HERSHEY_COMPLEX , 0.8 , (0,0,0) , 2)
            x += 28
        
        cv.imshow('frame', cv.cvtColor(frame , cv.COLOR_RGB2BGR))
    else:
        frame[0:pelak.shape[0] , 0:pelak.shape[1] , ...] = pelak
        cv.imshow('frame' , cv.cvtColor(frame , cv.COLOR_RGB2BGR))
    
    if cv.waitKey(1) & 0xff == ord('q'):
        break

cv.destroyAllWindows()
video.release()